<a href="https://colab.research.google.com/github/filippozuddas/ML-SRT-SETI/blob/main/notebooks/train_CVAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install setigen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.3/249.3 kB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 MB 32.8 MB/s eta 0:00:00
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.2.3
    Uninstalling pyparsing-3.2.3:
      Successfully uninstalled pyparsing-3.2.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pydot 3.0.4 requires pyparsing>=3.0.9, but you have pyparsing 2.4.7 which is incompatible.


In [27]:
import sys
import os

# --- CONFIGURAZIONE PERCORSI ---
PROJECT_ROOT = "/content/filippo/ML-SRT-SETI"
PROJECT_ROOT_LOCAL = "/home/filippo/TirocinioSETI/ML-SRT-SETI"

if not os.path.exists(PROJECT_ROOT_LOCAL):
    raise FileNotFoundError(f"{PROJECT_ROOT_LOCAL} folder not found")

os.chdir(PROJECT_ROOT_LOCAL)

if PROJECT_ROOT_LOCAL not in sys.path:
    sys.path.append(PROJECT_ROOT_LOCAL)

print(f"✅ Setup completed!")
print(f"   Working directory: {os.getcwd()}")

✅ Setup completed!
   Working directory: /home/filippo/TirocinioSETI/ML-SRT-SETI


In [28]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from src import config, models
from src.models import VAE

# Verifica GPU
print(f"TensorFlow Version: {tf.__version__}")
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

TensorFlow Version: 2.10.1
Num GPUs Available:  0


In [ ]:
DATASET_PATH = "data/synthetic_dataset.npz"
MODELS_FOLDER = "models"
MODELS_WEIGHTS_PATH = "models/weights/beta_vae_weights.h5"

os.makedirs(MODELS_FOLDER, exist_ok=True)

INPUT_SHAPE = (16, 512, 1)
LATENT_DIM = 8
ALPHA = 10.0
BETA = 1.5
BATCH_SIZE = 32
LR = 0.001
EPOCHS = 50

In [30]:
def create_tf_dataset(recon_data, true_data, false_data, batch_size):
    """
    Crea un tf.data.Dataset ottimizzato a partire dai tensori numpy
    """
    # Creazione del dataset da slice di tensori
    # Input X: (Reconstruction, True Cadence, False Cadence)
    # Target Y: Reconstruction
    ds = tf.data.Dataset.from_tensor_slices(
        ((recon_data, true_data, false_data), recon_data)
    )
    
    # Shuffle, Batching e Prefetching
    # drop_remainder=True è importante per training multi-GPU (MirroredStrategy)
    ds = ds.shuffle(2048).batch(batch_size, drop_remainder=True)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    
    return ds

def load_data(filepath, split=0.8):
    """
    Carica i dati dal file .npz, esegue lo split e restituisce i dataset TF pronti.
    """
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"File {filepath} non trovato!")
        
    print(f"Loading data from {filepath}...")
    with np.load(filepath) as f:
        mixed = f['mixed_cadence']
        true_c = f['true_cadence']
        false_c = f['false_cadence']
        
    print(f"Total samples: {len(mixed)}")
    
    # Prepariamo l'input per la ricostruzione (Solo il primo frame ON della sequenza)
    # Assumiamo che mixed sia (N, 6, 16, 512, 1) -> x_recon diventa (N, 16, 512, 1)
    x_recon = mixed[:, 0, :, :, :] 
    
    # Creazione degli indici per lo split
    indices = np.arange(len(mixed))
    train_idx, val_idx = train_test_split(indices, train_size=split, random_state=42)
        
    # Creazione Train Dataset
    # Passiamo direttamente le slice degli array usando gli indici di train
    train_ds = create_tf_dataset(
        recon_data=x_recon[train_idx],
        true_data=true_c[train_idx],
        false_data=false_c[train_idx],
        batch_size=BATCH_SIZE
    )
    
    # Creazione Validation Dataset
    # Passiamo direttamente le slice degli array usando gli indici di val
    val_ds = create_tf_dataset(
        recon_data=x_recon[val_idx],
        true_data=true_c[val_idx],
        false_data=false_c[val_idx],
        batch_size=BATCH_SIZE
    )
    
    return train_ds, val_ds

In [31]:
train_dataset, val_dataset = load_data(DATASET_PATH)

Loading data from data/synthetic_dataset.npz...
Total samples: 128


In [ ]:
strategy = tf.distribute.MirroredStrategy()

with strategy.scope(): 
    encoder = models.build_encoder(latent_dim=LATENT_DIM)
    decoder = models.build_decoder(latent_dim=LATENT_DIM)
    vae = VAE(encoder, decoder, alpha=ALPHA, beta=BETA)

    # Compilazione con ottimizzatore standard
    vae.compile(optimizer=keras.optimizers.Adam(learning_rate=LR))
    

tensorflow      WARNING  There are non-GPU devices in `tf.distribute.Strategy`, not using nccl allreduce.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
tensorflow      INFO     Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [25]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_total_loss", 
        patience=10, 
        restore_best_weights=True,
        verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_total_loss",
        factor=0.5,
        patience=5,
        verbose=1
    ),
]

In [26]:
print("Starting training...")

history = vae.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS,
    callbacks=callbacks
)

Starting training...


2025-11-24 16:17:16.648890: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:776] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_4"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
input: "Placeholder/_2"
input: "Placeholder/_3"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_DOUBLE
      type: DT_DOUBLE
      type: DT_DOUBLE
      type: DT_DOUBLE
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 102
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024TensorSliceDataset:8"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 16
        }
        dim {
          size: 512
        }
        dim {
          size: 1
        }
      }
      shape {
        dim {
          size: 6
       

Epoch 1/50


ValueError: Unexpected result of `train_function` (Empty logs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.